#Import Dependencies

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.optimizers
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt


In [ ]:
print(tf.__version__)

2.8.2


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Check what GPU is attatched to this session
To activate GPU:
-> Runtime -> Change Runtime Type -> Hardware Accelerator = GPU

In [ ]:
!nvidia-smi

Sat Aug 20 13:05:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Importing and Checking Data

In [80]:
DEAD_FILE_PATH = '/content/gdrive/MyDrive/train/dead/'
ALIVE_FILE_PATH = '/content/gdrive/MyDrive/train/alive/'

TRAIN_FILE_PATH = '/content/gdrive/MyDrive/train/'
TEST_FILE_PATH = '/content/gdrive/MyDrive/test/'
VALIDATION_FILE_PATH = '/content/gdrive/MyDrive/validation/'

In [81]:
img = image.load_img( DEAD_FILE_PATH +'Screenshot 2022-08-20 at 08.57.39.png')
plt.imshow(img)

FileNotFoundError: ignored

In [84]:
for filename in os.listdir(DEAD_FILE_PATH):
  full_path = os.path.join(DEAD_FILE_PATH, filename)
  if 'DS_Store' in filename or '.ipynb' in filename:
    print('DS_Store / .ipnyb file is not supposed to be here, skipping')
  else:
    print(filename)
    print(full_path)
    print(cv2.imread(full_path).shape)

Screenshot 2022-08-20 at 09.00.15.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.00.15.png
(900, 1742, 3)
Screenshot 2022-08-20 at 09.01.21.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.01.21.png
(1080, 1762, 3)
Screenshot 2022-08-20 at 09.01.41.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.01.41.png
(1122, 1726, 3)
Screenshot 2022-08-20 at 09.01.58.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.01.58.png
(520, 964, 3)
Screenshot 2022-08-20 at 09.03.27.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.03.27.png
(1040, 1702, 3)
Screenshot 2022-08-20 at 09.26.15.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.26.15.png
(1436, 2208, 3)
Screenshot 2022-08-20 at 09.30.41.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.30.41.png
(1086, 1962, 3)
Screenshot 2022-08-20 at 09.31.50.png
/content/gdrive/MyDrive/train/dead/Screenshot 2022-08-20 at 09.31.50.png
(12

#Normalise data and format


In [85]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [86]:
train_dataset = train.flow_from_directory(TRAIN_FILE_PATH,
                                          target_size = (500,500),
                                          class_mode = 'binary',
                                          batch_size = 3)

Found 68 images belonging to 2 classes.


In [90]:
validation_dataset = validation.flow_from_directory(VALIDATION_FILE_PATH,
                                          target_size = (500,500),
                                          class_mode = 'binary',
                                          batch_size = 3)

Found 9 images belonging to 2 classes.


Check that the alive and dead are mapped to the same value for test and train data

In [91]:
print(train_dataset.class_indices)
print(validation_dataset.class_indices)

{'alive': 0, 'dead': 1}
{'alive': 0, 'dead': 1}


#Generate Model

In [92]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5), strides = (2,2), padding = 'valid', input_shape = (500,500, 3))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (5,5), strides = (1,1), padding = 'valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1), padding = 'valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 248, 248, 32)      2432      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 124, 124, 32)     0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 120, 120, 64)      51264     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 60, 60, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 58, 58, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 29, 29, 128)     

In [93]:
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])

In [94]:
model_fit = model.fit(train_dataset, epochs = 25, validation_data = validation_dataset, steps_per_epoch=3)

Epoch 1/25
3/3 [==============================] - 2s 644ms/step - loss: 0.2295 - binary_accuracy: 0.8889 - val_loss: 10.1019 - val_binary_accuracy: 0.6667
Epoch 2/25
3/3 [==============================] - 2s 659ms/step - loss: 3.6512 - binary_accuracy: 0.8889 - val_loss: 6.8134 - val_binary_accuracy: 0.6667
Epoch 3/25
3/3 [==============================] - 2s 917ms/step - loss: 5.3948 - binary_accuracy: 0.4444 - val_loss: 0.8278 - val_binary_accuracy: 0.3333
Epoch 4/25
3/3 [==============================] - 1s 605ms/step - loss: 1.7261 - binary_accuracy: 0.4444 - val_loss: 2.5688 - val_binary_accuracy: 0.6667
Epoch 5/25
3/3 [==============================] - 2s 624ms/step - loss: 0.0470 - binary_accuracy: 1.0000 - val_loss: 0.4661 - val_binary_accuracy: 0.8889
Epoch 6/25
3/3 [==============================] - 2s 642ms/step - loss: 0.3555 - binary_accuracy: 0.7778 - val_loss: 1.6658 - val_binary_accuracy: 0.6667
Epoch 7/25
3/3 [==============================] - 1s 578ms/step - loss: 0.0

In [97]:
img = image.load_img(TEST_FILE_PATH + 'Screenshot 2022-08-20 at 15.54.09.png')
image_array = image.img_to_array(img)
expanded_image_array = np.expand_dims(image_array, axis = 0)
images = np.vstack([expanded_image_array])
resized_image = tf.image.resize(
    images,
    size = (500,500))
prediction = model.predict(resized_image)
print(prediction)
if prediction == [[1.]]:
  print('Dead forrest')
else:
  print('Alive forrest')

[[0.]]
Alive forrest


In [ ]:
img = image.load_img(TEST_FILE_PATH + 'Screenshot 2022-08-20 at 15.54.09.png')